# Summary
Here, we combine all impact layers to create a composite human impact layers.

# Setting up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

# Import layers

In [ ]:
from google.cloud import storage
def list_blobs(bucket_name, prefix):
    storage_client = storage.Client.from_service_account_json(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))

    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)
    blob_names = []

    for blob in blobs:
      if blob.name != prefix:
        blob_names.append('gs://' + bucket_name + '/' + blob.name)

    return blob_names

built_blobs = list_blobs('nature-watch-bucket', 'COGS/built/2023/')
built = geemap.load_GeoTIFFs(built_blobs).mosaic()

crop_blobs = list_blobs('nature-watch-bucket', 'COGS/crop/2019/')
crop = geemap.load_GeoTIFFs(crop_blobs).mosaic()

fire_blobs = list_blobs('nature-watch-bucket', 'COGS/fire/2022/')
fire = geemap.load_GeoTIFFs(fire_blobs).mosaic()

mines_features = ee.FeatureCollection("projects/sat-io/open-datasets/global-mining/global_mining_polygons")
mines = mines_features.reduceToImage(
  properties=['AREA'],
  reducer=ee.Reducer.median()
).gt(0).selfMask().select(['median'], ['label'])

treeloss = ee.Image('UMD/hansen/global_forest_change_2021_v1_9').select('lossyear');


# Reproject to 1km2 pixels

In [ ]:
project_proj = fire.projection()


built_resampled = built.reproject(project_proj, None, 30).reduceResolution(
      reducer = ee.Reducer.sum().unweighted(),
      maxPixels=10000
      ).reproject(project_proj, None, 1000)

In [ ]:
built_resampled.getInfo()

In [ ]:
block_list = list(range(1, 26))

blocks = []
with open("/content/drive/MyDrive/data/blocks.geojson") as f:
    json_data = json.load(f)
    for block_id in block_list:
        for feature in json_data['features']:
            if feature['properties']['id'] == block_id:
                feature['properties']['style'] = {}
                blocks.append(feature)

ee_blocks = [geemap.geojson_to_ee(block) for block in blocks]

# Export

In [ ]:
# Loop through blocks and export
for id, block in enumerate(ee_blocks):
  if id != 13:
    image_name = 'built2023_' + str(id)
    fileNamePrefix = 'COGS/built/2023/' + image_name


    exportConfig = {
        'image': built,
        'description': image_name,
        'bucket': 'nature-watch-bucket',
        'fileNamePrefix': fileNamePrefix,
        'scale': 30,
        'maxPixels': 3867395000,
        'region': block,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {'cloudOptimized': True}
    }

    task = ee.batch.Export.image.toCloudStorage(**exportConfig)
    task.start()

In [ ]:
# Or just export one block
id = 19
block = ee_blocks[id]


image_name = 'built2023_' + str(id)
fileNamePrefix = 'COGS/built/2023/' + image_name


exportConfig = {
    'image': built,
    'description': image_name,
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': fileNamePrefix,
    'scale': 30,
    'maxPixels': 3867395000,
    'region': block,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
}

task = ee.batch.Export.image.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(built_resampled, {}, 'built_resampled')

Map.setCenter(31.47, -24.98, 12)
Map